# Continuous control
## DDPG algorithm implementation

---

This self sufficient contains the whole code needed for an implementation of the DDPG reinforcement learning algorithm in the "reacher environment".

We follow the implementation described in the paper "Continuous control with reinforcement learning": https://arxiv.org/abs/1509.02971

In [1]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from collections import namedtuple, deque, defaultdict
import random
from matplotlib import pyplot as plt
import pickle
from IPython import display
import pylab as pl
import copy
import time
import os

%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Depending on which version of the environment you want to run, select one line or the other in the cell below:

In [2]:
def seed_everything(seed=924):   #911
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
# seed_everything()

In [3]:
# Select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='/Users/manuelsh/code/deep-reinforcement-learning/p2_continuous-control/Reacher.app')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/Users/manuelsh/code/deep-reinforcement-learning/p3_collab-compet/Tennis.app',
                       no_graphics = True)
#env = UnityEnvironment(file_name="/data/Tennis_Linux_NoVis/Tennis")

# Get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


### Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_actors = len(env_info.agents)
print('Number of agents:', num_actors)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### Implementation of relevant classes
In the following cells we implement the ReplayBuffer class (coming from the Udacity repo), Ornstein Uhlenbeck noise generation, soft-update function (which updates the parameters of the target function) and the classes for the actor (`DeterministicPolicy`) and the critic (`QFunction`).

In [5]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", 
                                                                "action", 
                                                                "reward", 
                                                                "next_state", 
                                                                "done",
                                                                "td_error"])
        self.td_errors = []
    
    def add(self, states, actions, rewards, next_states, dones, td_errors):
        """Add a new experience to memory."""
#         for state, action, reward, next_state, done, td_error  in zip(states, actions, rewards, 
#                                                                       next_states, dones, td_errors):
        e = self.experience(states, actions, rewards, next_states, dones, td_errors)
        self.memory.append(e)
        self.td_errors.append(max(td_errors))
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        
        td_error_probs = self.get_td_errors_probs(self.td_errors)
        elements = np.random.choice(range(len(self.memory)), size=BATCH_SIZE, 
                                    p=td_error_probs, replace=False)
        experiences = [self.memory[element] for element in elements]
        #from IPython.core.debugger import Tracer; Tracer()()
        states = torch.from_numpy(np.stack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.stack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.stack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.stack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.stack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        td_errors = torch.from_numpy(np.stack([e.td_error for e in experiences if e is not None])).float().to(device)
  
        return (states, actions, rewards, next_states, dones, td_errors)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)
        
    def get_td_errors_probs(self, td_errors):
        td_error_probs_scaled = np.abs(td_errors)**PE_ALPHA
        return td_error_probs_scaled / (np.abs(self.td_errors)**PE_ALPHA).sum()


In [6]:
class OUNoise:
    """Ornstein-Uhlenbeck process."""

    def __init__(self, size, mu=0., theta=0.2, sigma=1):
        """Initialize parameters and noise process."""
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.state = copy.copy(self.mu)
        self.size = size

    def sample(self):
        """Update internal state and return it as a noise sample."""
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.standard_normal(self.size)
        self.state = x + dx
        return self.state

In [7]:
def soft_update(local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)


In [8]:
class QFunction(nn.Module):
    def __init__(self, state_dim, actions_dim, hidden_units=(256, 128), activation=F.relu):
        super(QFunction, self).__init__()
        dims = (state_dim + actions_dim, ) + hidden_units + (1, )
        self.layers = nn.ModuleList(
            [nn.Linear(dim_in, dim_out) for dim_in, dim_out in zip(dims[:-1], dims[1:])])
        self.activation = activation      
        
    def forward(self, state, action):
        x = torch.cat([state, action], dim=1)
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        return self.layers[-1](x)

In [ ]:
class DeterministicPolicy(nn.Module):
    def __init__(self, state_dim, actions_dim, hidden_units=(256, 128) , activation=F.relu):
        super(DeterministicPolicy, self).__init__()
        dims = (state_dim,) + hidden_units +(actions_dim,)
        self.layers = nn.ModuleList(
            [nn.Linear(dim_in, dim_out) for dim_in, dim_out in zip(dims[:-1], dims[1:])])
        self.activation = activation      
        
    def forward(self, state):
        x = state
        for layer in self.layers[:-1]:
            x = self.activation(layer(x))
        return F.tanh( self.layers[-1](x) )

In [ ]:
def perform_an_action(states, noise):
        # perform an action
        states_tensor = torch.FloatTensor(states).to(device)
        with torch.no_grad():
            actions = np.stack([actors[i](states_tensor[i]).detach().cpu().numpy() + noise.sample() \
                                     for i in range(num_actors)])
            if ACTION_CLIPPING:
                actions = np.clip(actions, -1, 1)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations / STATES_DIVISOR
        rewards = env_info.rewards                         
        dones = env_info.local_done
        return states, actions, rewards, next_states, dones      

In [ ]:
def watch_env(t_max=600, ou_sigma=0):
    time.sleep(0.1)
    env_info = env.reset(train_mode=False)[brain_name]      # reset the environment    
    states = env_info.vector_observations / STATES_DIVISOR                # get the current state (for each agent)
    scores = np.zeros(num_actors)                          # initialize the score (for each agent)
    noise = OUNoise(size=action_size, theta=ou_sigma, sigma=OU_SIGMA_END)
    noise_sample = noise.sample()
    for i in range(t_max):
        states, actions, rewards, next_states, dones = perform_an_action(states, noise)
        scores += rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        time.sleep(0.01)
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over actors) this episode: {}'.format(np.mean(scores)))

In [ ]:
def adapt_c(tensor):
    # adapts dimensions for the critic
    if len(tensor.shape)==2:
        tensor = tensor.unsqueeze(0)
    return tensor.view(-1, tensor.shape[-1]*num_actors)

In [ ]:
def get_sum_grads(model):
    return np.sum( [ float( torch.abs(p.grad).sum() ) for p in model.parameters() ] )

def get_grads_proportion(model):
    return np.mean( [ float( torch.abs(p.grad/p).mean() ) for p in model.parameters() ] )

def get_param_mean(model):
    return np.mean( [ float( torch.abs(p).mean() ) for p in model.parameters() ] )

### Parameters, instantiation and training loop

In [ ]:
# Main parameters (some of them will be overwritten in the grid search):

BUFFER_SIZE = int(1e6)
BATCH_SIZE = 128      
LR_CRITIC = 1E-3
LR_ACTOR = 1E-3

GAMMA = 0.99
TAU = 0.008

UPDATE_STEPS = 1
NUM_UPDATES = 5

OU_SIGMA_START = 1
OU_SIGMA_END = 0.
OU_THETA = 0.15
EPIS_SIGMA_END = 500 # number of episodes to set sigma to zero

PE_ALPHA = 0#0.7
PE_BETA_START = 0#0.5 # prioritised experience replay beta start
PE_BETA_END = 0 # prioritised experience replay beta end
EPIS_PE_BETA_END = 1500
ACTION_CLIPPING = True

STATES_DIVISOR = 30  #divide states to normalise data

In [ ]:
# Saving grid search results
monitoring_gs = {}
total_scores_gs = {}

# monitoring_gs = pickle.load(open('monitoring.pickle', 'rb'))
# total_scores_gs = pickle.load(open('total_scores_gs.pickle', 'rb'))

In [ ]:
# Instantiation
seeds = [23,21,6243, 928, 1929, 1, 5, 9, 888, 666, 15, 44, 134, 43678, 8376]
ou_sigma_starts = [3, 2, 1]
epis_sigma_ends = [300, 500, 1000]
ou_thetas = [ 0.2]
gammas = [ 0.99]
taus = [0.008, 0.001, 0.0005]
update_steps = [1,2,5,10]
learning_rates_actor = [ 1e-2, 1e-3 ]
learning_rates_critic = [1e-4, 1e-3]
grad_clipping_values = [10, 1, 0.5, 0.1]
trials = 100

for trial in range(trials):
    
    # Select randomly
    already_done = True
    while already_done:
        # Parameters initialization
        seed = np.random.choice(seeds)
        OU_SIGMA_START = np.random.choice(ou_sigma_starts)
        EPIS_SIGMA_END = np.random.choice(epis_sigma_ends)
        OU_THETA = np.random.choice(ou_thetas)
        GAMMA = np.random.choice(gammas)
        TAU = np.random.choice(taus)
        UPDATE_STEPS = np.random.choice(update_steps)
        NUM_UPDATES = np.random.choice(range(1, UPDATE_STEPS+5))
        LR_ACTOR = np.random.choice(learning_rates_actor)
        LR_CRITIC = np.random.choice(learning_rates_critic)
        CRITIC_GRAD_CLIPPING = np.random.choice(grad_clipping_values)
        point = (seed, OU_SIGMA_START,EPIS_SIGMA_END, OU_THETA, 
                 GAMMA, TAU, UPDATE_STEPS, NUM_UPDATES, LR_ACTOR, LR_CRITIC,CRITIC_GRAD_CLIPPING )
        if point not in monitoring_gs.keys():
            already_done = False
                                   
    # Instantiate and seed
    seed_everything(seed)
    critics = [QFunction(state_size*num_actors, 
                         action_size*num_actors).to(device) for _ in range(num_actors)]
    actors = [DeterministicPolicy(state_size, action_size).to(device) for _ in range(num_actors)]

    critic_targets = [QFunction(state_size*num_actors, 
                                action_size*num_actors).to(device) for _ in range(num_actors)]
    actor_targets = [DeterministicPolicy(state_size, action_size).to(device) for _ in range(num_actors)]
    
    for i in range(num_actors):
        critic_targets[i].load_state_dict(critics[i].state_dict())
        actor_targets[i].load_state_dict(actors[i].state_dict())
        
    critic_optimizers = [torch.optim.Adam(critics[i].parameters(), lr=LR_CRITIC) for i in range(num_actors)]
    actor_optimizers = [torch.optim.Adam(actors[i].parameters(), lr=LR_ACTOR) for i in range(num_actors)]

    replay_buffer = ReplayBuffer(action_size=action_size, buffer_size=BUFFER_SIZE, batch_size=BATCH_SIZE)
    
    # Initialization of needed variables
    total_scores = []
    total_scores_queue = deque(maxlen=100)
    running_averages = []
    monitoring = defaultdict(list)

    num_episodes = 4000
    step_counter = 0

    for episode in range(num_episodes):
        env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
        states = env_info.vector_observations / STATES_DIVISOR                 # get the current state (for each actor)
        scores = np.zeros(num_actors)                          # initialize the score (for each actor)
        ou_sigma= OU_SIGMA_START - (OU_SIGMA_START - OU_SIGMA_END) * min(1., episode/EPIS_SIGMA_END)
        noise = OUNoise(action_size, sigma=ou_sigma, theta=OU_THETA)
        beta = PE_BETA_START - (PE_BETA_START - PE_BETA_END) * min(1., episode/EPIS_PE_BETA_END)
        
        while True:
            states, actions, rewards, next_states, dones = perform_an_action(states, noise)
            next_states_tensor = torch.FloatTensor(next_states).to(device)
            states_tensor = torch.FloatTensor(states).to(device)
            actions_tensor = torch.FloatTensor(actions).to(device)
            rewards_tensor = torch.FloatTensor(rewards).to(device)
            dones_tensor =  torch.FloatTensor(dones).to(device)
            
            # Calculate td_errors
            next_actions_all = torch.stack([actor_targets[i](next_states_tensor[i]) for i in range(num_actors)])
            
            critics_target_value = [critic_targets[i](adapt_c(next_states_tensor), 
                                                      adapt_c(next_actions_all))\
                                    .squeeze() * (1-dones_tensor[i]) for i in range(num_actors)]

            td_errors = [float((rewards_tensor[i] + critics_target_value[i]*GAMMA - \
                         critics[i](adapt_c(states_tensor),
                                    adapt_c(actions_tensor)).squeeze())) for i in range(num_actors)]

            # Store in replay buffer
            replay_buffer.add(states=states, 
                              actions=actions, 
                              rewards=rewards, 
                              next_states=next_states, 
                              dones=dones,
                              td_errors=td_errors)
            scores += rewards                         # update the score (for each actor)
            states = next_states                    # roll over states to next time step 
            step_counter+=1  
            
            if (len(replay_buffer.memory)>BATCH_SIZE) & (step_counter%UPDATE_STEPS==0) :
                for _ in range(NUM_UPDATES):
                    # Train the critic
                    states_r, actions_r, rewards_r, next_states_r, dones_r, td_errors_r = replay_buffer.sample()
                    
                    weights = 1/ ( replay_buffer.get_td_errors_probs(td_errors_r) * len(replay_buffer))**beta
                    for i in range(num_actors):
                        next_actions_all_r = torch.stack([actor_targets[j](next_states_r[:,j,:]) \
                                                          for j in range(num_actors)])
                        critic_target_result = critic_targets[i](adapt_c(next_states_r), 
                                                                 adapt_c(next_actions_all_r)).squeeze() * (1-dones_r[:,i])

                        y = (rewards_r[:,i] + critic_target_result * GAMMA) * weights[:,i].to(device)
                        y = y.detach()
                        
                        # Train critic
                        critic_result = critics[i](adapt_c(states_r), adapt_c(actions_r)).squeeze()
                        critic_loss_value = F.mse_loss(critic_result, y)
                        critic_optimizers[i].zero_grad()
                        critic_loss_value.backward()
                        torch.nn.utils.clip_grad_norm_(critics[i].parameters(), CRITIC_GRAD_CLIPPING)
                        critic_optimizers[i].step()

                        # Train the actor
                        actions_all_r = torch.cat([actors[j](states_r[:,j,:]) if (j==i) else\
                                                   actors[j](states_r[:,j,:]).detach() for j in range(num_actors)])
                        actor_loss_value = -critics[i](adapt_c(states_r), adapt_c(actions_all_r)).mean()
                        actor_optimizers[i].zero_grad()
                        actor_loss_value.backward()
                        actor_optimizers[i].step()

                        # Update parameters
                        soft_update(actors[i], actor_targets[i], TAU)
                        soft_update(critics[i], critic_targets[i], TAU)

                        monitoring[f'critic_target_result_{i}'].append(float(critic_target_result.mean()))
                        monitoring[f'y_{i}'].append(float(y.mean()))
                        monitoring[f'critic_result_{i}'].append(float(critic_result.mean()))
                        monitoring[f'gradient_critic_{i}'].append(get_sum_grads(critics[i]))
                        monitoring[f'gradient_actor_{i}'].append(get_sum_grads(actors[i]))
                        monitoring[f'gradient_prop_critic_{i}'].append(get_grads_proportion(critics[i]))
                        monitoring[f'gradient_prop_actor_{i}'].append(get_grads_proportion(actors[i]))
                        monitoring[f'param_mean_critic_{i}'].append(get_param_mean(critics[i]))
                        monitoring[f'param_mean_actor_{i}'].append(get_param_mean(actors[i]))


            if np.any(dones):                                  # exit loop if episode finished
                break

        # Store scores
        total_scores.append(np.max(scores))
        total_scores_queue.append(np.max(scores))
        running_average = np.mean(total_scores_queue)
        running_averages.append(running_average)
        monitoring['total_scores_mean'].append(np.mean(scores))
        
        # Save variables
        pickle.dump(total_scores, open(f'total_scores_seed_{seed}.pickle','wb'))
        pickle.dump(running_average, open(f'running_average_seed_{seed}.pickle','wb'))
        pickle.dump(monitoring, open(f'monitoring_seed_{seed}.pickle','wb'))
        
                
        # Store scores grid search
        total_scores_gs[point] = total_scores
        monitoring_gs[point] = monitoring
        pickle.dump(total_scores_gs, open('total_scores_gs.pickle','wb'))
        pickle.dump(monitoring_gs, open('monitoring_gs.pickle','wb'))

        # Plot
        display.clear_output(wait=True)

    #     pl.figure(1)
    #     pl.plot(total_scores, label='Episode score')
    #     pl.plot(running_averages, label='Running average score, 100 episodes', linestyle='--')
    #     display.display(pl.gcf())

    #     pl.figure(2)
    #     for key in monitoring.keys():
    #         pl.plot(monitoring[key], label=key)
    #     display.display(pl.gcf())

        print(f'Point: {point} \n (seed, OU_SIGMA_START,EPIS_SIGMA_END, OU_THETA, GAMMA, TAU, UPDATE_STEPS, NUM_UPDATES, LR_ACTOR, LR_CRITIC,CRITIC_GRAD_CLIPPING )')
        print(f'Running average score {len(total_scores)}: {running_average}')
        print(f'Max score achieved: {np.max(total_scores)}')
        print(f'Running average score (real): {np.mean(monitoring["total_scores_mean"][-100:])}')
        if len(monitoring['critic_result'])>0:
            print(f'Last Q estimations (running mean): {np.mean(monitoring["critic_result"][-100:])}')
#         if running_average>=0.5:
#             print(f'Problem solved at episode {len(total_scores)}')
#             break
        
        if (running_average == 0.) & (len(running_averages)>1000):
            break
            
#         if (running_average < 0.005) & (len(running_averages)>700):
#             break
                  
#         if (running_average < 0.01) & (len(running_averages)>2000):
#             break

    #     if len(total_scores)%25==0:
    #         watch_env()

Point: (9, 3, 300, 0.2, 0.99, 0.001, 2, 3, 0.001, 0.001, 0.5) 
 (seed, OU_SIGMA_START,EPIS_SIGMA_END, OU_THETA, GAMMA, TAU, UPDATE_STEPS, NUM_UPDATES, LR_ACTOR, LR_CRITIC,CRITIC_GRAD_CLIPPING )
Running average score 430: 0.0
Max score achieved: 0.10000000149011612
Running average score (real): -0.004999999888241291


In [ ]:
# Point: (5, 3,1000, 0.2, 0.99, 0.0005, 2, 2, 0.01, 0.0001, 1) seems to learn
# (seed, OU_SIGMA_START,EPIS_SIGMA_END, OU_THETA, GAMMA, TAU, UPDATE_STEPS, NUM_UPDATES, LR_ACTOR, LR_CRITIC, CRITIC_GRAD_CLIPPING)
# gradient clipping?
# normalisation of input features?
# initialization of nn?
# ver qué acción toma despues de

### Saves parameters of model

In [ ]:
for i in range(num_actors):
    torch.save(actors[i].state_dict(), f'models-parameters/actor_{i}.torch')
    torch.save(critics[i].state_dict(), f'models-parameters/critic_{i}.torch')
    torch.save(actor_targets[i].state_dict(), f'models-parameters/actor_target_{i}.torch')
    torch.save(critic_targets[i].state_dict(), f'models-parameters/critic_target_{i}.torch')

In [ ]:
num_actors = 2
critics = [QFunction(state_size*num_actors, 
                     action_size*num_actors).to(device) for _ in range(num_actors)]
actors = [DeterministicPolicy(state_size, action_size).to(device) for _ in range(num_actors)]
for i in range(num_actors):
    actors[i].load_state_dict(torch.load(f'models-parameters/actor_{i}.torch'))
    critics[i].load_state_dict(torch.load(f'models-parameters/critic_{i}.torch'))

In [ ]:
watch_env()

In [ ]:
# pl.figure(figsize=(15,6))
# for key in monitoring.keys():
#     pl.plot(monitoring[key], label=key, alpha=0.6)
# pl.legend()
# pl.grid()

pl.figure(figsize=(15,6))
pl.plot(total_scores, label='Episode score')
pl.plot(running_averages, label='Running average score, 100 episodes', linestyle='--')
pl.legend()
pl.grid()